# CloudEx Algo Main Script

#### Default Imports

In [1]:
import sys 
sys.path

new_paths = ['/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/root/CloudExchange', '/root/']
for p in new_paths : 
    sys.path.append(p) 


In [2]:
sys.path

['/root/cs349f/implementation1',
 '/root/.pyenv/versions/3.7.7/lib/python37.zip',
 '/root/.pyenv/versions/3.7.7/lib/python3.7',
 '/root/.pyenv/versions/3.7.7/lib/python3.7/lib-dynload',
 '',
 '/home/steam1994/.local/lib/python3.7/site-packages',
 '/root/.pyenv/versions/3.7.7/lib/python3.7/site-packages',
 '/home/steam1994/.local/lib/python3.7/site-packages/IPython/extensions',
 '/home/steam1994/.ipython',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/root/CloudExchange',
 '/root/']

In [3]:
# Import packages.
import datetime
import json
import os
import sys
import time

import pandas as pd
import numpy as np
import redis
from pandas.core.common import SettingWithCopyWarning

import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# CloudEx imports.
import cloud_ex

# Import AlgorithmicTrader helper class.
from algorithmic_trader_shay import AlgorithmicTrader
from algorithmic_trader_shay import summarize_historical_trades_df

# Start Redis and its Python API.
os.system("redis-server --daemonize yes")
time.sleep(1)

# Get CloudEX and VM-specific config. 
# NOTE: gateway_ip will be null when the exchange is not online 
def get_vm_config():
    with open("/root/vm_config.json", "r") as read_file:
        config = json.load(read_file)
    return config

config = get_vm_config()


# utilities  

ORDER_FIELDS_LIST = [
    'Symbol', 'OrderID', 'CancelID', 'ClientID', 'OrderType', 'OrderAction',
    'SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
    'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice', 'ResultType','NumShares'
]

TRADE_FIELDS_LIST = [
    "Symbol", "BuyerSerialNum", "SellerSerialNum", "BuyerOrderID",
    "SellerOrderID", "BuyerClientID", "SellerClientID", "ExecPrice",
    "CashTraded", "SharesTraded", "CreationTimestamp", "ReleaseTimestamp",
    "TradeSerialNum"
]

'''
Takes in a cloud_ex.VectorOrder with serialized orders and returns a DataFrame
'''
def OrderDF(order_vec):
    if not len(order_vec):
        return pd.DataFrame(columns=ORDER_FIELDS_LIST)
    df = pd.DataFrame(order_vec).applymap(lambda x:x.SerializeOrder())[0].str.split('|', expand=True)
    df.columns = ORDER_FIELDS_LIST
    for label in ['SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
                  'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice','NumShares']:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df

'''
Takes in a cloud_ex.VectorOrder with serialized trades and returns a DataFrame
'''
def TradeDF(trade_vec):
    if not len(trade_vec):
        return pd.DataFrame(columns=TRADE_FIELDS_LIST)
    df = pd.DataFrame(trade_vec).applymap(lambda x:x.SerializeTrade())[0].str.split('|', expand=True)
    df.columns = TRADE_FIELDS_LIST
    for label in ["ExecPrice", "CashTraded", "SharesTraded",
                  "CreationTimestamp", "ReleaseTimestamp", "TradeSerialNum"]:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df

'''
Takes in a cloud_ex.MapStringOrder mapping Order ID strings to outstanding the coorresponding orders, 
and returns a DataFrame
'''
def OutstandingOrderDF(outstanding_orders):
    if not len(outstanding_orders):
        return pd.DataFrame(columns=ORDER_FIELDS_LIST)
    df = (pd.DataFrame(outstanding_orders.items())[1]).apply(lambda x:x.SerializeOrder()).str.split('|', expand=True)
    df.columns = ORDER_FIELDS_LIST
    for label in ['SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
                  'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice','NumShares']:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df 

In [4]:
config

{'gateway_ip': '10.128.15.220',
 'client_id': 'C15',
 'client_token': 'dyjpqwzyvminsvjpxtyasxdnzpnqkuux',
 'project_id': 'jinkun-pro1',
 'bigtable_id': 'test-bt',
 'table_name': 'record-tb'}

#### Custom Imports

In [5]:
import importlib 
import sys 
#importlib.reload(sys.modules['mean_reversion_shay'])
def reload(r) : 
    importlib.reload(sys.modules[r])

In [6]:
import strategies_shay   
import threading
import utilities as u 
default_symbols = ['AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'AZ', 'BA', 'BB', 'BC', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM', 'BN', 'BO', 'BP', 'BQ', 'BR', 'BS', 'BT', 'BU', 'BV', 'BW', 'BX', 'BY', 'BZ', 'CA', 'CB', 'CC', 'CD', 'CE', 'CF', 'CG', 'CH', 'CI', 'CJ', 'CK', 'CL', 'CM', 'CN', 'CO', 'CP', 'CQ', 'CR', 'CS', 'CT', 'CU', 'CV', 'CW', 'CX', 'CY', 'CZ', 'DA', 'DB', 'DC', 'DD', 'DE', 'DF', 'DG', 'DH', 'DI', 'DJ', 'DK', 'DL', 'DM', 'DN', 'DO', 'DP', 'DQ', 'DR', 'DS', 'DT', 'DU', 'DV']
default_top_symbols = ['DV', 'CQ', 'DN', 'CL', 'CT', 'BF', 'CW', 'AW', 'DS', 'CS']

In [7]:
trader = None

#### Get the trader object

In [44]:
def getTrader() : 
    global trader
    if trader : 
        u.debug("Returning saved trader")
        return trader 

    # Get relevant fields from VM-specific config. Token is yours only, so don't make it public.
    gateway_ip = config["gateway_ip"]
    client_id = config["client_id"]
    client_token = config["client_token"]

    # Clear any existing data locally.
    redis_api = redis.Redis()
    redis_api.flushall();

    # Create CloudEx base trader object.
    trader = cloud_ex.Trader(gateway_ip, client_id, client_token)
    return trader 

def getSymbols() : 
    trader = getTrader() 
    return trader.GetSymbols()


def getPortfolio(): 
    portfolio_mat = cloud_ex.MapStringInt()
    trader.GetPortfolioMatrix(portfolio_mat)
    return portfolio_mat
    

In [53]:
portfolio_mat = cloud_ex.MapStringInt()
trader.GetPortfolioMatrix(portfolio_mat)

True

In [54]:
portfolio_mat['CC']

0

In [9]:
T = getTrader()

In [10]:
config['gateway_ip']

'10.128.15.220'

#### Main Todos



### Selecting symbols to trade on based on aggregate volume over last n seconds  

We need to figure out which symbols to trade on prior to the initiation of trading. It seems natural to rank them by some metric and then take the top N of them, for example volume 

In [11]:
import numpy as np

def aggregate_volume(symbol, seconds_in_past) : 
    """
    Gets the most recent 'seconds_in_past' historical data for the symbol and compute the total 
    'CashTraded' 
    """
    u.debug("Getting aggregate volume for {}".format(symbol))
    end_time_ms = int(time.time()*1e3)
    start_time_ms = end_time_ms - int(seconds_in_past*1e3)
    symbol_trades_vec = cloud_ex.VectorTrade()  
    cloud_ex.MarketDataAPI.PullTrades(config['project_id'], config['bigtable_id'], 
                                          config['table_name'], symbol, start_time_ms, 
                                          end_time_ms, symbol_trades_vec)
    sym_df = TradeDF(symbol_trades_vec)
    #print(sym_df)
    print("Returning volume for symbol:" + symbol)
    return np.sum(sym_df['CashTraded'])    

def periodicity_metric(symbol, seconds_in_past, normalize_with_dc = True): 
    """
    Find symbols with high degrees of periodicity.
    """
    u.debug("Getting periodicity of {}".format(symbol))
    end_time_ms = int(time.time()*1e3)
    start_time_ms = end_time_ms - int(seconds_in_past*1e3)
    symbol_trades_vec = cloud_ex.VectorTrade()  
    cloud_ex.MarketDataAPI.PullTrades(config['project_id'], config['bigtable_id'], 
                                          config['table_name'], symbol, start_time_ms, 
                                          end_time_ms, symbol_trades_vec)
    sym_df = TradeDF(symbol_trades_vec)
    sym_df.to_csv("periodicity_" + str(symbol) + "_" + str(time.time()) +".csv", sep=',')
    close_price_vector = sym_df['ExecPrice']

    ps = np.abs(np.fft.fft(close_price_vector))**2
    freqs = np.fft.fftfreq(len(close_price_vector), GLOBALS['BIN_INTERVAL_MS']/10**3)
    if normalize_with_dc:
        periodicity = np.max(ps[1:])/float(np.sum(ps))
    else:
        periodicity = np.max(ps[1:])/float(np.sum(ps[1:]))

    print(symbol, periodicity)
    return periodicity, freqs

def multithreaded_volume_request(syms,seconds_in_past) : 
    import concurrent.futures
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [] 
        for sym in syms : 
            print("Submitting thread for sym: " + sym )
            future = executor.submit(aggregate_volume, sym, seconds_in_past)
            futures.append(future)
        
    # now we get the results 
    values = [future.result() for future in futures ] 
    return values 
    
def get_volume_for_symbol_in_thread(symbol, seconds_in_past) : 
    # create the thread 
    t = threading.Thread(target=aggregate_volume,
                         args=(symbol,seconds_in_past))
    # start the thread 
    t.start() 
    # return it 
    return t 


def ranked_volume_symbols(symbols , seconds_in_past) : 
    
    ## -- 
    threads = [ get_volume_for_symbol_in_thread(symbol,seconds_in_past) for symbol in symbols  ] 
    
    for thread in threads : 
        thread.join() 
    
    
    ## -- 

def rank_symbols_by_volume(symbols , seconds_in_past) : 
    data = [ [symbol, aggregate_volume(symbol,seconds_in_past)  ] for  symbol in symbols ] 
    data.sort(key=lambda x:  x[1] , reverse=True )
    return data

def get_top_n_symbols_by_volume(symbols,seconds_in_past, n )  : 
    ranked = rank_symbols_by_volume(symbols,seconds_in_past) 
    syms = [ x[0] for x in ranked[0:n]] 
    return (syms , ranked)  

def rank_symbols_by_periodicity(symbols , seconds_in_past) : 
    data = [ [symbol, periodicity_metric(symbol, seconds_in_past)  ] for  symbol in symbols ] 
    data.sort(key=lambda x:  x[1][0], reverse=True )
    return data

def get_top_n_symbols_by_periodicity(symbols,seconds_in_past, n )  : 
    ranked = rank_symbols_by_periodicity(symbols, seconds_in_past) 
    syms = [ x[0] for x in ranked[0:n]] 
    return (syms , ranked)

In [29]:
def place_order(trader, symbol, price, num_shares, buy=True,limit=True):
        """
        Place an order.

        :param symbol: Symbol to buy or sell (str).
        :param price: Price at which to execute the order (int).
        :param num_shares: Number of shares to buy or sell (int).
        :param buy: Whether to buy or sell (bool).
        """
    
        returned_order_ = cloud_ex.Order()
    
        if limit : 
            type_ = cloud_ex.OrderType.limit 
        else : 
            type_ = cloud_ex.OrderType.market  

        action_ = cloud_ex.OrderAction.buy if buy else cloud_ex.OrderAction.sell

        # Submit order and wait
        result = trader.SubmitOrder(symbol, returned_order_, type_,
                                         action_, num_shares, int(price))
        if result != cloud_ex.OrderResult.in_sequencer:
            return None
        return returned_order_.order_id_

In [51]:
place_order(T,'CC', 250, 7768, buy=False)

'CC.0002501603941920347211500212015'

In [52]:
outstanding_orders()

You have 1 outstanding orders.


,Symbol,OrderID,CancelID,ClientID,OrderType,OrderAction,SubmitTimestamp,GatewayTimestamp,EnqueueTimestamp,DequeueTimestamp,OrderSerialNum,LimitPrice,ResultType,NumShares
0,CD,CD.0999991603938586161236500115815,NULL,C15,L,B,1603938586161011,1603938586161243,1603938586161404,1603938586161593,2884,1,V,1


In [21]:
periodicity_metric("CC",60*18,False)


Getting periodicity of CC
CC 0.20331264862536685


(0.20331264862536685,
 array([ 0.        ,  0.00115674,  0.00231348, ..., -0.00347021,
        -0.00231348, -0.00115674]))

In [27]:
syms = ['AE', 'AN','AB', 'AI' ,'AO'] 
seconds = 60*60

ts1= time.time()
multithreaded_volume_request(syms, seconds)
te1= time.time() 

ts2= time.time() 
for sym in syms : 
    aggregate_volume(sym,seconds)
te2= time.time() 

print("Multithreaded took {} seconds".format(te1-ts1))
print("Singlethreaded took {} seconds".format(te2-ts2))


Submitting thread for sym: AE
Getting aggregate volume for AE
Submitting thread for sym: AN
Getting aggregate volume for AN
Submitting thread for sym: AB
Getting aggregate volume for AB
Submitting thread for sym: AI
Getting aggregate volume for AI
Submitting thread for sym: AO
Getting aggregate volume for AO
Returning volume for symbol:AEReturning volume for symbol:AN

Returning volume for symbol:AI
Returning volume for symbol:AB
Returning volume for symbol:AO
Getting aggregate volume for AE
Returning volume for symbol:AE
Getting aggregate volume for AN
Returning volume for symbol:AN
Getting aggregate volume for AB
Returning volume for symbol:AB
Getting aggregate volume for AI
Returning volume for symbol:AI
Getting aggregate volume for AO
Returning volume for symbol:AO
Multithreaded took 81.81353855133057 seconds
Singlethreaded took 78.71345901489258 seconds


#### Conclusion from above is that multithreaded market data request is not helpful 

In [31]:
top_symbols, symbol_ranks = get_top_n_symbols_by_volume(default_symbols, 60*3, 10)

Getting aggregate volume for AA
Returning volume for symbol:AA
Getting aggregate volume for AB
Returning volume for symbol:AB
Getting aggregate volume for AC
Returning volume for symbol:AC
Getting aggregate volume for AD
Returning volume for symbol:AD
Getting aggregate volume for AE
Returning volume for symbol:AE
Getting aggregate volume for AF
Returning volume for symbol:AF
Getting aggregate volume for AG
Returning volume for symbol:AG
Getting aggregate volume for AH
Returning volume for symbol:AH
Getting aggregate volume for AI
Returning volume for symbol:AI
Getting aggregate volume for AJ
Returning volume for symbol:AJ
Getting aggregate volume for AK
Returning volume for symbol:AK
Getting aggregate volume for AL
Returning volume for symbol:AL
Getting aggregate volume for AM
Returning volume for symbol:AM
Getting aggregate volume for AN
Returning volume for symbol:AN
Getting aggregate volume for AO
Returning volume for symbol:AO
Getting aggregate volume for AP
Returning volume for sy

KeyboardInterrupt: 

In [13]:
aggregate_volume('AD', 60*4)

Getting aggregate volume for AD
Returning volume for symbol:AD


1149880

In [13]:
most_periodic, periodicity_ranks = get_top_n_symbols_by_periodicity(default_symbols, 24*60*60, 10)

Getting periodicity of AA
AA 0.055483397042988034
Getting periodicity of AB
AB 0.06965702415101696
Getting periodicity of AC
AC 0.10059764228088328
Getting periodicity of AD
AD 0.036987538507577994
Getting periodicity of AE
AE 0.14946302581715132
Getting periodicity of AF
AF 0.10715947003989401
Getting periodicity of AG
AG 0.04199406921324091
Getting periodicity of AH
AH 0.033995320903498105
Getting periodicity of AI
AI 0.08467866496358983
Getting periodicity of AJ
AJ 0.07998298830498775
Getting periodicity of AK
AK 0.01763063118307809
Getting periodicity of AL
AL 0.0912894781879455
Getting periodicity of AM
AM 0.006977975242788612
Getting periodicity of AN
AN 0.07369052539832958
Getting periodicity of AO
AO 0.08522728107083319
Getting periodicity of AP
AP 0.004067258003619044
Getting periodicity of AQ
AQ 0.01983633845710913
Getting periodicity of AR
AR 0.06461503821977443
Getting periodicity of AS
AS 0.08893484888577335
Getting periodicity of AT
AT 0.09464903894037947
Getting periodic

In [20]:
most_periodic

['AB', 'DC', 'AU', 'AC', 'BH', 'BS', 'AL', 'AQ', 'AK', 'BL']

In [21]:
periodicity_ranks

[['AB',
  (0.0021347010313830864,
   array([ 0.        ,  0.0025974 ,  0.00519481,  0.00779221,  0.01038961,
           0.01298701,  0.01558442,  0.01818182,  0.02077922,  0.02337662,
           0.02597403,  0.02857143,  0.03116883,  0.03376623,  0.03636364,
           0.03896104,  0.04155844,  0.04415584,  0.04675325,  0.04935065,
           0.05194805,  0.05454545,  0.05714286,  0.05974026,  0.06233766,
           0.06493506,  0.06753247,  0.07012987,  0.07272727,  0.07532468,
           0.07792208,  0.08051948,  0.08311688,  0.08571429,  0.08831169,
           0.09090909,  0.09350649,  0.0961039 ,  0.0987013 ,  0.1012987 ,
           0.1038961 ,  0.10649351,  0.10909091,  0.11168831,  0.11428571,
           0.11688312,  0.11948052,  0.12207792,  0.12467532,  0.12727273,
           0.12987013,  0.13246753,  0.13506494,  0.13766234,  0.14025974,
           0.14285714,  0.14545455,  0.14805195,  0.15064935,  0.15324675,
           0.15584416,  0.15844156,  0.16103896,  0.16363636,  0.1

In [14]:
getSymbols()

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. cloud_ex.Trader(arg0: str, arg1: str, arg2: str)

Invoked with: None, 'C15', 'dyjpqwzyvminsvjpxtyasxdnzpnqkuux'

In [41]:
top_symbols

['CX', 'CZ', 'AH', 'DA', 'AA', 'AD', 'CS', 'AG', 'AO', 'AP']

In [36]:
symbol_ranks

[['CX', 1810092],
 ['CZ', 1755790],
 ['AH', 1250054],
 ['DA', 1223190],
 ['AA', 952611],
 ['AD', 833049],
 ['CS', 791593],
 ['AG', 612493],
 ['AO', 491351],
 ['AP', 398533],
 ['AL', 397636],
 ['AC', 360296],
 ['AB', 324539],
 ['AE', 257818],
 ['DU', 244409],
 ['DC', 233574],
 ['AQ', 222745],
 ['AM', 137258],
 ['AT', 131034],
 ['BG', 102394],
 ['CT', 69741],
 ['DN', 67146],
 ['AF', 66522],
 ['AS', 65161],
 ['DV', 64797],
 ['AK', 59043],
 ['DO', 58409],
 ['BA', 58294],
 ['CU', 57585],
 ['DR', 57445],
 ['DS', 55721],
 ['BB', 55399],
 ['AW', 54619],
 ['DD', 52422],
 ['AX', 51844],
 ['CW', 51681],
 ['AZ', 50923],
 ['DB', 50808],
 ['BF', 49356],
 ['DT', 49302],
 ['DI', 48363],
 ['DE', 47453],
 ['BE', 46780],
 ['CR', 45690],
 ['DF', 45351],
 ['DH', 45262],
 ['DG', 45033],
 ['CV', 44878],
 ['DK', 44838],
 ['DJ', 42193],
 ['AY', 41792],
 ['BH', 41535],
 ['BK', 40999],
 ['BI', 39919],
 ['CY', 39208],
 ['AV', 38277],
 ['BD', 38029],
 ['BM', 37681],
 ['BC', 36847],
 ['DQ', 36776],
 ['BT', 35077],


### QUERY ORDER HISTORY

In [31]:
### Order Monitoring 
def outstanding_orders() : 
    outstanding_orders = cloud_ex.MapStringOrder()
    trader.GetOutstandingOrders(outstanding_orders)
    u.debug("You have {} outstanding orders.".format(len(outstanding_orders)))
    # Transform outstanding orders into a DataFrame
    outstanding_orders = OutstandingOrderDF(outstanding_orders)
    return outstanding_orders
    
def historical_orders() : 
    my_historical_orders = cloud_ex.VectorOrder()
    trader.GetAllHistoricalOrders(my_historical_orders)
    u.debug("You have submitted a total of {} order(s).".format(len(my_historical_orders))) 
    my_historical_orders_df = OrderDF(my_historical_orders)
    return my_historical_orders_df

def historical_trades() : 
    my_historical_trades = cloud_ex.VectorTrade()
    trader.GetAllHistoricalTrades(my_historical_trades)
    u.debug("You have made a total of {} trade(s).".format(len(my_historical_trades)))
    my_historical_trades_df = TradeDF(my_historical_trades)
    return my_historical_trades_df

In [34]:
outstanding_orders()

You have 1 outstanding orders.


,Symbol,OrderID,CancelID,ClientID,OrderType,OrderAction,SubmitTimestamp,GatewayTimestamp,EnqueueTimestamp,DequeueTimestamp,OrderSerialNum,LimitPrice,ResultType,NumShares
0,CD,CD.0999991603938586161236500115815,NULL,C15,L,B,1603938586161011,1603938586161243,1603938586161404,1603938586161593,2884,1,V,1


In [35]:
historical_trades()

You have made a total of 3719 trade(s).


,Symbol,BuyerSerialNum,SellerSerialNum,BuyerOrderID,SellerOrderID,BuyerClientID,SellerClientID,ExecPrice,CashTraded,SharesTraded,CreationTimestamp,ReleaseTimestamp,TradeSerialNum
0,DN,1,0,DN.0996001603767967486441500000015,NULL,C15,NULL,400,400,1,1603767967488426,1603767967488776,473058
1,DN,2,0,DN.0996011603767969100024500000115,NULL,C15,NULL,400,400,1,1603767969100400,1603767969100750,473246
2,DN,3,0,DN.0996001603767969662163500000215,NULL,C15,NULL,399,399,1,1603767969662528,1603767969662878,473298
3,DN,4,0,DN.0996021603767970239776500000315,NULL,C15,NULL,399,399,1,1603767970240146,1603767970240496,473364
4,DN,5,0,DN.0996011603767970809166500000415,NULL,C15,NULL,399,399,1,1603767971698254,1603767971698604,473514
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3714,CC,0,3844,NULL,CC.0002501603941380012154500211815,NULL,C15,207,20700,100,1603941380012538,1603941380012888,2088444
3715,CC,0,3844,NULL,CC.0002501603941380012154500211815,NULL,C15,207,20700,100,1603941380012543,1603941380012893,2088445
3716,CC,0,3844,NULL,CC.0002501603941380012154500211815,NULL,C15,207,20700,100,1603941380012548,1603941380012898,2088446
3717,CC,0,3844,NULL,CC.0002501603941380012154500211815,NULL,C15,207,139932,676,1603941380012550,1603941380012900,2088447


# 1. Backtesting

Let's see how we can backtest our trading algorithms to get them ready for live trading. In the following cells we will download historical data and evaluate how well a mean reversion trader would have done.

In [65]:
GLOBALS = { 
    'NUM_SHARES' :  1 , 
    'BIN_INTERVAL_MS' : 500,  #interval to bin the data with 
    'WAIT_INTERVAL_SECONDS' : 0.5, 
    'BACKTEST_LOOKBACK_PERIOD_SECONDS' : 6*60  , #amount of historical data to backtest on 
    'MAX_NUM_ORDERS' : 2*60*2 , #how long the algo will trade for, in # of bins 
} 

## 1.1 Get our bank of strategies

### Note on strategy and strategy parameters terminology

A strategy is implemented with specific parameters. <b>The pair of the strategy and parameters will be called an "algo"</b>
The backtest logic will take a dictionary of string keys (identifiers) to a tuple of (strategy , params). This dictionary will be called "algo_bank" 

For example => 

In [64]:
# NOte a fully specified strategy is called an "algo" and consists of a tuple of (strategy, kwargs) where kwargs provide
# the parameters for the strategy 

to_reload = ["algorithmic_trader_shay", 
             "mean_reversion_shay",
             "momentum_shay", 
             "random_buy_shay", 
             "random_sell_shay", 
             "buy_strategy_shay", 
             "sell_strategy_shay", 
             "strategies_shay" ] 

for r in to_reload : 
    reload(r) 
    
def mean_reversion_algo(ma,t) : 
    # Helper function for making the "algo" data structure
    # NOTE this returns a TUPLE of (strategy , kwargs)
    return (strategies_shay.strategies['mean_reversion'].strategy, { 'ma' : ma, 'threshold' : t} ) 

def momentum_algo(p1,p2,t) : 
    # NOTE this returns a TUPLE of (strategy , kwargs)
    return (strategies_shay.strategies['momentum'].strategy, { 'p1' : p1, 'p2' : p2, 'threshold' : t} ) 

def random_buy_algo(p) : 
    return (strategies_shay.strategies['random_buy'].strategy , { 'p' : p} ) 

def random_sell_algo(p) : 
    return (strategies_shay.strategies['random_sell'].strategy , { 'p' : p} ) 



# Define an "Algo Bank", which holds the (strategy, kwargs) pair, indexed by a unique identifier 
algo_bank = { 
     #'mr_10_3' : mean_reversion_algo(10,1),
     #'mr_10_5' : mean_reversion_algo(10,5), 
     #'mr_10_10' : mean_reversion_algo(10,10),
     #'mo_5_5_1' : momentum_algo(0.5,0.5,1) , 
     #'mo_5_5_3' : momentum_algo(0.5,0.5,3) , 
     #'mo_8_2_1' : momentum_algo(0.8,0.2,1) , 
     #'mo_8_2_3' : momentum_algo(0.8,0.2,3) , 
    'buy'    : random_buy_algo(1) , 
    'sell'    : random_sell_algo(1) , 
 
}


def backtest_strategy_with_symbol(data,strategy, symbol, params,algoname ) :  
    """
    Backtests a given strategy, kwarg pair on a diven symbol using the last seconds_in_past seconds of historical data 
    """
   
    
    #bin_interval_ms = 500
    #summarize_historical_trades_df(symbol_historical_trades_df, bin_interval_ms)
    
    # Initial capital and shares we will use for the backtest. 
    init_capital = 100000
    init_shares = 1000

    # Set up trading algorithm parameters.
    num_shares = 10

    # Set up the strategy 
    trader = None
    algo = strategy(trader, [symbol],GLOBALS['BIN_INTERVAL_MS'])  # instantiates the strategy which is subclass of algorithmic_trader

    # Run the backtest.
    if algoname in ['buy','sell'] : 
        
        
        closes = data['ExecPrice']
        roi = ( closes.iloc[-1] - np.mean(closes) ) / np.mean(closes)
        if algoname == 'sell' : 
            roi = roi * -1 
            
        action_list = [] 
        
    else : 
        roi, action_list = algo.backtest(data,
                     num_shares, 
                     init_capital, 
                     init_shares, 
                     **params)

    u.debug("Algo ROI={}%".format(roi))
    return roi,action_list
    
    

def backtest_algobank_on_symbols(algobank, symbols) : 
    """
    Backtests all algos ( strategy, kwarg pairs) on each symbol in symbols, over the last seconds_in_past seconds 
    of data 
    
    Returns the results in a sorted list with algos ranked by roi 
    """
    results = [] 
    
    
    for symbol in symbols : 
        
        u.debug("\n\nBacktesting symbol: {}".format(symbol) ) 
        
        end_time_ms = int(time.time()*1e3)
        start_time_ms = end_time_ms - int(GLOBALS['BACKTEST_LOOKBACK_PERIOD_SECONDS']*1e3)
        symbol_trades_vec = cloud_ex.VectorTrade()  
        cloud_ex.MarketDataAPI.PullTrades(config['project_id'], config['bigtable_id'], 
                                          config['table_name'], symbol, start_time_ms, 
                                          end_time_ms, symbol_trades_vec)
        u.debug("There are a total of {} trades for {} symbol".format(len(symbol_trades_vec), symbol))
        symbol_historical_trades_df = TradeDF(symbol_trades_vec)
        symbol_historical_trades_df = symbol_historical_trades_df.sort_values(by="CreationTimestamp")
    
        for (algoname,algo) in algobank.items() : 
            strategy, params = algo
            u.debug("algo={}".format(algoname))
            roi,action_list = backtest_strategy_with_symbol(symbol_historical_trades_df,strategy, symbol, params,algoname)
            results.append( [  roi, symbol, algoname,action_list ])
            u.debug("Made {} trades with roi: {}".format(len(action_list), roi))
    
    results.sort(key=lambda x : x[0], reverse=True) 
    return results         

            

In [55]:
end_time_ms = int(time.time()*1e3)
start_time_ms = end_time_ms - int(60*30*1e3)
for symbol in ['CC'] : 
    symbol_trades_vec = cloud_ex.VectorTrade()  
    cloud_ex.MarketDataAPI.PullTrades(config['project_id'], config['bigtable_id'], 
                                            config['table_name'], symbol, start_time_ms, 
                                            end_time_ms, symbol_trades_vec)
    u.debug("There are a total of {} trades for {} symbol".format(len(symbol_trades_vec), symbol))
    symbol_historical_trades_df = TradeDF(symbol_trades_vec)
    df = symbol_historical_trades_df.sort_values(by="CreationTimestamp")
    df.to_csv("market_data_{}_to_train.csv".format(symbol))


There are a total of 2748 trades for CC symbol


# Deploying the trading threads 


In [71]:
ACTIVE_THREADS = [] # global handle on the currently deployed threads 

In [68]:
def run_and_evaluate_algorithm(**kwargs) : 
    """
    Intended as target of new thread() 
    1. Launches the algo on the symbol and starts trading 
    2. should calculate ROI of the algo 
    4. Writes ROI and submitted order ids to disk 
    """
    

    name     = kwargs['name']
    strategy = kwargs['strategy']  
    strategy_parameters = kwargs['strategy_parameters'] 
    num_shares = kwargs['num_shares'] 
    max_num_orders = kwargs['max_num_orders'] 
    symbol   = kwargs['symbol']
    trader   = kwargs['trader'] #reference to the 1 trader instance connnected to cloudX
    
    
    print("RE: Running algo: {} - {}".format(name,symbol)) 

    
    # create the AlgorithmicTrader Object 
    algo = strategy(trader, [symbol], bin_interval_ms=GLOBALS['BIN_INTERVAL_MS']) 
    
    # get and set id for this trader  (for logging purposes)
    trader_id = name + "_" + str(time.time()).split(".")[0]
    #print(trader_id) 
    #print(type(algo))
    #algo.super().set_id(trader_id)  #added this method 
    
    # Calculate portfolio state pre-trading 
    pass 

    # start and finish trading
    order_ids = algo.trade(symbol,num_shares,max_num_orders,GLOBALS['WAIT_INTERVAL_SECONDS'] ,trader_id=trader_id,**strategy_parameters)
    # can SIMULATE if we want 
    
    # calculate portfolio state post-trading
    pass 

    #time.sleep(10)
    # write the order ids to a log file 
    u.logfile(trader_id + "_order_ids", json.dumps(order_ids) )
    # write the ROI information a log file 
    pass     

def run_algorithm_in_thread(kwargs) : 
    # create the thread 
    print("Running algo in thread: {} - {}".format(kwargs['name'],kwargs['symbol']))
    t = threading.Thread(target=run_and_evaluate_algorithm,
                         kwargs=kwargs)
    # start the thread 
    t.start() 
    # return it 
    return t 

def deploy_top_N_algorithms(trader,ranked_algos, N,  num_shares, max_num_orders) : 
    global ACTIVE_THREADS
    sublist = ranked_algos[0:N] 
    print("Deploying algos: ")
    print(sublist) 
    
    # potential fix for the multi-threading?? 
    active_symbols = [ x[1] for x in sublist ] 
    print("Setting trader active symbols to: {}".format(json.dumps(active_symbols)))
    trader.set_active_symbols( active_symbols )
        
    ts = []
    for to_deploy in sublist : 
        roi, symbol, algoname , _  =  to_deploy 
        strategy, strategy_parameters = algo_bank[algoname]  
        
        arguments = { 
            'name' : algoname, 
            'strategy' : strategy, 
            'strategy_parameters' : strategy_parameters, 
            'num_shares' : num_shares, 
            'max_num_orders' : max_num_orders,  
            'symbol' : symbol , 
            'trader' : trader ,
        }
        ts.append(run_algorithm_in_thread(arguments))
    ACTIVE_THREADS = ts 
    return ts 


def trade_algorithmically(trader,algobank,symbols,N=5) : 
    from random import sample 

    
    while True : 
        sym_subset = sample(symbols,25)
        u.logfile("tradeloop", "\nBacktesting on symbols =>")
        u.logfile("tradeloop", "\n{}\n".format(json.dumps(sym_subset)))

        u.logfile("tradeloop", "\n{}, Doing backtest".format(time.time()))
        ranked = backtest_algobank_on_symbols(algobank,sym_subset)
        
        fname = "backtest_results_{}".format(time.time())
        u.logfile(fname, json.dumps(ranked))
        u.logfile("tradeloop","backtest_results=>")
        u.logfile("tradeloop", "\n{}\n".format(json.dumps(ranked[0:N])))
        
        u.logfile("tradeloop", "\n{}, Launching Trading".format(time.time()))
        
        
        algo_threads = deploy_top_N_algorithms(trader,ranked,N, GLOBALS['NUM_SHARES'] , GLOBALS['MAX_NUM_ORDERS']) 
        for t in algo_threads : 
            t.join() 
    
          
def stop_trading_threads() : 
        print("There were {} threads running".format(len(threading.enumerate())))
        import os 
        import time 
        os.environ['STOP_TRADING'] = "TRUE" 
        time.sleep(GLOBALS['WAIT_INTERVAL_SECONDS']+0.1)
        print("There are now {} threads running".format(len(threading.enumerate())))
        os.environ['STOP_TRADING'] = "FALSE" 

In [ ]:
trade_algorithmically(getTrader(), algo_bank,default_symbols,N=1)

Returning saved trader


Backtesting symbol: BZ
There are a total of 388 trades for BZ symbol
algo=buy
Running in offline mode. Could not set active symbols.
Algo ROI=-0.015707433155480382%
Made 0 trades with roi: -0.015707433155480382
algo=sell
Running in offline mode. Could not set active symbols.
Algo ROI=0.015707433155480382%
Made 0 trades with roi: 0.015707433155480382


Backtesting symbol: AS
There are a total of 340 trades for AS symbol
algo=buy
Running in offline mode. Could not set active symbols.
Algo ROI=0.003979971755039122%
Made 0 trades with roi: 0.003979971755039122
algo=sell
Running in offline mode. Could not set active symbols.
Algo ROI=-0.003979971755039122%
Made 0 trades with roi: -0.003979971755039122


Backtesting symbol: CO
There are a total of 341 trades for CO symbol
algo=buy
Running in offline mode. Could not set active symbols.
Algo ROI=-0.033113578069894185%
Made 0 trades with roi: -0.033113578069894185
algo=sell
Running in offline mode. Could not set active 

Exception in thread Thread-21:
Traceback (most recent call last):
  File "/root/.pyenv/versions/3.7.7/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/root/.pyenv/versions/3.7.7/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-65-beda5a3e423f>", line 35, in run_and_evaluate_algorithm
    order_ids = algo.trade(symbol,num_shares,max_num_orders,GLOBALS['WAIT_INTERVAL_SECONDS'] ,trader_id=trader_id,**strategy_parameters)
  File "/root/cs349f/implementation1/algorithmic_trader_shay.py", line 190, in trade
    order_id = self.place_order(symbol, price, num_shares, buy=buy)
  File "/root/cs349f/implementation1/algorithmic_trader_shay.py", line 98, in place_order
    action_, num_shares, int(price))
IndexError: basic_string::substr: __pos (which is 2) > this->size() (which is 1)





Backtesting symbol: DS
There are a total of 328 trades for DS symbol
algo=buy
Running in offline mode. Could not set active symbols.
Algo ROI=0.005318220785822096%
Made 0 trades with roi: 0.005318220785822096
algo=sell
Running in offline mode. Could not set active symbols.
Algo ROI=-0.005318220785822096%
Made 0 trades with roi: -0.005318220785822096


Backtesting symbol: CD
There are a total of 782 trades for CD symbol
algo=buy
Running in offline mode. Could not set active symbols.
Algo ROI=-0.045781533198886976%
Made 0 trades with roi: -0.045781533198886976
algo=sell
Running in offline mode. Could not set active symbols.
Algo ROI=0.045781533198886976%
Made 0 trades with roi: 0.045781533198886976


Backtesting symbol: DB
There are a total of 259 trades for DB symbol
algo=buy
Running in offline mode. Could not set active symbols.
Algo ROI=0.002354466336850715%
Made 0 trades with roi: 0.002354466336850715
algo=sell
Running in offline mode. Could not set active symbols.
Algo ROI=-0.0023

In [ ]:
#historical_trades()